# Introduction

In these three exercises you will get an introduction to the functionality of the new pyRSA-toolbox. For illustration these exercises use simulated RDMs from the paper "Inferring brain-computational mechanisms with models of activity measurements" by Kriegeskorte & Diedrichsen (2016). These data are generated by applying different measurement models to the layers of Alexnet, the deep neural network model, which sparked the interest in deep learning. Thus, we have noise free model RDMs for each layer, which represent different measurement models, i.e. models how the representation is distorted by the measurement. Additionally, we have simulated fMRI data with different amounts of noise added. 

Our overall aim in this setting is to infer which representation the data rdms were based on, i.e. which layer was used for generating the data. Towards this aim we will make three steps:

In *Exercise 1*, we will load the data, convert them into the formats used in the toolbox and have a first exploratory look at the data.

In *Exercise 2*, we will compare the RDMs based on the undistorted representations to the measured RDMs. This is the classical and simplest approach and already allows us to perform model comparisons and the general evaluation of model-RDMs. We will see that this does not allow us to correctly infer the underlying representation, because the measurement process distorts the RDMs too much.

In *Exercise 3*, we will apply flexible models, which allow the models to choose the measurement model which best explains the data. To evaluate such flexible models additional cross-validation is necessary, which we also discuss in this exercise.

# Exercise 1: Data and RDM handling

In [ ]:
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
import pyrsa

## Load model RDMs
Here the models are different layers of Alexnet.
For each layer, different models of how the fMRI voxels sample the neurons are being considered.

The simulated data were generated in Matlab (Kriegeskorte & Diedrichsen 2016). Thus, we load the Matlab files in .mat format.

For each model-RDM, we obtain a model name which is the layer used to generate RDM, a measurement model, which specifies the applied distortions and the actual rdm, which was saved as a vector for each RDM:

In [ ]:
matlab_data = io.matlab.loadmat('rdms_inferring/modelRDMs.mat')
matlab_data = matlab_data['modelRDMs']
n_models = len(matlab_data[0])
model_names = [matlab_data[0][i][0][0] for i in range(n_models)]
measurement_model = [matlab_data[0][i][1][0] for i in range(n_models)]
rdms_array = np.array([matlab_data[0][i][2][0] for i in range(n_models)])

These steps are not specific to the toolbox, but to the format the RDMs were originally saved in.
To load other data, simply transform them such that they are numpy arrays of either the whole RDM or vector format of the upper triangular part of the matrix.

## Store the model RDMs as a pyRSA object
We place the RDMs in a pyRSA object which can contain additional descriptors for the RDMs and the experimental conditions.
Here we label each RDM with the name of the brain-computational model (AlexNet layer) and the name of the measurement model.

In [ ]:
model_rdms = pyrsa.rdm.RDMs(rdms_array,
                            rdm_descriptors={'brain_computational_model':model_names,
                                             'measurement_model':measurement_model}
                           )

model_rdms is now a custom object, which contains all the rdms from the .mat file with the additional information.
It also has a few functions for forming subsets of the data, saving and loading, etc.

## Show the RDMs from AlexNet layer conv1

As a simple example select the rdms, which correspond to the first convolutional layer. These can then be plotted using the function pyrsa.vis.show_rdm.

In [ ]:
conv1_rdms = model_rdms.subset('brain_computational_model','conv1')
plt.figure(figsize=(10,10))
pyrsa.vis.show_rdm(conv1_rdms, do_rank_transform=True, rdm_descriptor='measurement_model')

These are the RDMs which were generated from convolutional layer 1 by different measurement models. Each RDM is labeled with the name of the measurement model. Also in the lower right corner the average RDM is plotted.

## Print info about a set of RDMs
The pyRSA objects can simply be passed to the print function to obtain a short description of their content.

In [ ]:
print(conv1_rdms)

## Questions

Of course, you can also show all RDMs or select any other subset. Have a look at the different RDMs!

How many RDMs are there for each layer?

Generate a plot which shows all RDMs with the 'complete' measurement model.

How different do the different measurement models look to you and how different do the different layers look?

# Exercise 2: Fixed model inference
## Load data RDMs
Here we use simulated data to demonstrate RSA inference.
Since we know the true data-generating model in each case, we can tell when inference fails or succeeds.

For each data RDM, we obtain the name of the underlying Layer, a full width half maximum (fhwm) value specifying how strongly the simulated voxels average the representation and a noise standard deviation, specifying how much noise was added to the voxel responses.

In [ ]:
matlab_data = io.matlab.loadmat('rdms_inferring/noisyModelRDMs_demo.mat')
repr_names_matlab = matlab_data['reprNames']
fwhms_matlab = matlab_data['FWHMs']
noise_std_matlab = matlab_data['relNoiseStds']
rdms_matlab = matlab_data['noisyModelRDMs']
repr_names = [repr_names_matlab[i][0][0] for i in range(repr_names_matlab.shape[0])]
fwhms = fwhms_matlab.squeeze().astype('float')
noise_std = noise_std_matlab.squeeze().astype('float')
rdms_matrix = rdms_matlab.squeeze().astype('float')


## Choose one of the data RDMs for inference

Here we choose which data RDMs we use for the exercise. You can change the representation, tthe noise level and the amount of averaging by chaning the index values at the beginning.

We then convert the chosen data RDMs into an pyrsa RDMs object and display them as we did for the model RDMs.

In [ ]:
# indices choosing brain-computational model, noise level, and the size of the kernel with which each voxel samples the neural activity
i_rep = 2 #np.random.randint(len(repr_names)) 
i_noise = 1 #np.random.randint(len(noise_std))
i_fwhm = 0 #np.random.randint(len(fwhms))

# print the chosen representation definition
repr_name = repr_names[i_rep]
print('The chosen ground truth model is:')
print(repr_name)
print('with noise level:')
print(noise_std[i_noise])
print('with averaging width (full width at half magnitude):')
print(fwhms[i_fwhm])

# put the rdms into an RDMs object and show it
rdms_data = pyrsa.rdm.RDMs(rdms_matrix[:, i_rep, i_fwhm, i_noise, :].transpose())
pyrsa.vis.show_rdm(rdms_data, do_rank_transform=True)


## Define fixed models
An "RDM model" is a pyRSA object that can predict a data RDM.
For example, an RDM model may contain a set of predictor RDMs, which predict the data RDM as a weighted combination.
Here we use fixed RDM models, which contain just a single RDM with no parameters to be fitted.

Models are generated by first choosing the right RDM in this case the one with the right "brain_computational_model" and the "measurement_model" "complete", which corresponds to no distortions added. This object is then passed to the function pyrsa.model.ModelFixed, which generates a fixed RDM model. These RDM models are then collected in the list models. 

In [ ]:
models = []
for i_model in np.unique(model_names):
    rdm_m = model_rdms.subset('brain_computational_model', i_model).subset('measurement_model','complete')
    m = pyrsa.model.ModelFixed(i_model, rdm_m)
    models.append(m)

print('created the following models:')
for i in range(len(models)):
    print(models[i].name)

## RSA 1.0: Just compare model RDMs to measured RDMs
evaluate models naively, i.e. simply compute the average correlation to the data RDMs

In [ ]:
results_1 = pyrsa.inference.eval_fixed(models, rdms_data, method='corr')
pyrsa.vis.plot_model_comparison(results_1)

#results_1 = pyrsa.inference.eval_fixed(models, rdms_data, method='spearman')
#pyrsa.vis.plot_model_comparison(results_1)

#results_1 = pyrsa.inference.eval_fixed(models, rdms_data, method='tau-a')
#pyrsa.vis.plot_model_comparison(results_1)

#results_1 = pyrsa.inference.eval_fixed(models, rdms_data, method='rho-a')
#pyrsa.vis.plot_model_comparison(results_1)

In these plots the models do not have errorbars as we did not run any estimate of the variablitiy.
The upper noise ceiling is computed by finding the RDM with the highest possible average similarity to the measured RDMs. This is not 1 because the RDMs for different subjects or measurements differ. The lower noise ceiling is a leave one out crossvalidation of this averaging procedure, i.e. we find the RDM to perform optimally on all but one of the RDMs and evaluate this average RDM on the left out RDM. Each RDM is left out once.

To perform statistical comparisons and estimate how uncertain we should be about the models' performance we can perform bootstrapping:

In each plot the errobars correspond to +/- one SEM based on the bootrap samples.
The lines above the plot show which pairwise comparisons are significant.

## Model comparison by bootstrapping the subjects
We can bootstrap resample the subjects, which estimates how variable the model performances would be if we sampled new subjects with the same stimuli. Based on that uncertainty estimate we can statistically compare model performances against each other. As we have many comparisons between models we perform FDR correction on those tests:

In [ ]:
results_2a = pyrsa.inference.eval_bootstrap_rdm(models, rdms_data, method='corr')
pyrsa.vis.plot_model_comparison(results_2a, plot_pair_tests='FDR')

## Model comparison by bootstrapping the stimuli
We can alternatively bootstrap resample the stimuli to estimate how much model performance would vary if we sampled the subjects we have with new stimuli. 

In [ ]:
results_2b = pyrsa.inference.eval_bootstrap_pattern(models, rdms_data, method='corr')
pyrsa.vis.plot_model_comparison(results_2b, plot_pair_tests='FDR')

## Model comparison by bootstrapping both stimuli and subjects
Finally, we can bootstrap resample both stimuli and subjects to estimate how variable the model performances would be if we measured new subjects with new stimuli:

In [ ]:
results_2c = pyrsa.inference.eval_bootstrap(models, rdms_data, method='corr')
pyrsa.vis.plot_model_comparison(results_2c, plot_pair_tests='FDR')

## Questions
Does the right model win? And do the mean estimates from bootstrapping differ from the evaluations over the whole dataset?

Compare the results for the different bootstrapping methods. Which method leads to the widest confidence intervals, which one to the smallest?

# Exercise 3: Crossvalidation for flexible models
## defining flexible models
Here we use selection models, i.e. each model layer gets a list of rdms and only specifies that one of those is the right one

In [ ]:
models_flex = []
for i_model in np.unique(model_names):
    models_flex.append(pyrsa.model.ModelSelect(i_model,
        model_rdms.subset('brain_computational_model', i_model)))

print('created the following models:')
for i in range(len(models_flex)):
    print(models_flex[i].name)

## Crossvalidation
As we are now using flexible models, we have to do crossvalidation to get an estimate how well the model would do on new unseen data.

As a first step we create the training test and ceil sets. The last one corresponds to the training data for the testset which is necessary for calculating a noise ceiling. 

In [ ]:
train_set, test_set, ceil_set = pyrsa.inference.sets_k_fold(rdms_data, k_pattern=3, k_rdm=2)


With these sets we can now evaluate our models as we did without crossvalidaton and plot the results. With a single training/testset combination we do not get errorbars.

In [ ]:
results_3_cv = pyrsa.inference.crossval(models_flex, rdms_data, train_set, test_set,
                                        ceil_set=ceil_set, method='corr')
# plot results
pyrsa.vis.plot_model_comparison(results_3_cv)

## Bootstrapped Crossvalidation

Finally, we can perform bootstrapping around the crossvalidation to get uncertainty estimates for the evaluation:

In [ ]:
results_3_full = pyrsa.inference.bootstrap_crossval(models_flex, rdms_data, k_pattern=3, k_rdm=2, method='corr', N=100)
# plot results
pyrsa.vis.plot_model_comparison(results_3_full, plot_pair_tests='FDR')

## Questions

Does the right model win?

Try some different settings for the bootstrap: How do the results change when you make the training and testsets larger or smaller?